In [1]:
import os
import sys
sys.path.append("..")

import tensorflow as tf

import cst_model as cst
import distortion_layers as ly
import base_models as bm
import callbacks as cb


# tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR) 

In [2]:
os.getcwd()

'/main_dir/felipe/projects/CST/notebooks'

In [3]:
class AdjSaturation(tf.keras.layers.Layer):
    """
    Layer to apply a random change in saturation to the image. Uses tf.image.random_saturation,
    which converts image to HSV, adjusts the S channel by a factor and changes back to RGB.
    A factor > 1 makes the image more saturated (i.e. stronger color), and < 1 makes it less
    saturated (i.e. grayer).

    Image must be RGB [0,255] (int or float). Outputs image [0,255] (float).

    Attributes:
        - lower: (float) min factor for saturation. has to be > 0.
        - upper: (float) max factor for saturation. has to be > lower
    """
    def __init__(self, value=1.):
        super(AdjSaturation, self).__init__()
        self.value = value

    def call(self,inputs, training=None):
        if training or training is None:
            return tf.image.adjust_saturation(inputs, self.value)
        return inputs

### Parameters


In [4]:
# train_path = "../data"
train_path = "/main_dir/felipe/projects/cp_toolbox_data/artifacts_project/training_data/v4"

tile_size = 128
batch_size = 32
channels = 3
n_st_components = 2
alpha = 10
epochs = 10
class_mode = "sparse"

validation_split = 0.2

### Create distortion layer

In [5]:
dcs = {
    "contrast": {"factor": [0.2, 0.2]},
    "color": {"factor": [20,0,20]},
    "blur": {"filter_shape": 2, "sigma": 5.},  # kernel size is 'filter_shape * 2 + 1'
    "brightness": {"lower": .85, "upper":1.15}
}

layers = [
    ly.RandomColorByChannel(**dcs["color"]), 
    tf.keras.layers.RandomContrast(**dcs["contrast"]),
    ly.RandomBrightness(**dcs["brightness"]),
    ly.RandomGaussianBlur(**dcs["blur"]),
]


# dist_layer = [AdjSaturation(10),
#               tf.keras.layers.Lambda(lambda x: x * 0. + 55)]

dist_layer = layers



2023-07-05 15:40:53.333340: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-05 15:40:53.340163: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-05 15:40:53.340360: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-05 15:40:53.341259: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

### Load images and assign parameters

In [6]:
gen = tf.keras.preprocessing.image.ImageDataGenerator(
    validation_split = validation_split
)

t_flow = gen.flow_from_directory(
    directory=train_path,
    target_size=(tile_size,tile_size),
    color_mode='rgb',  # rgb for color
    batch_size=batch_size,
    class_mode=class_mode,  # 'sparse' for multiclass, 'binary' for binary 
    subset='training'
)

v_flow = gen.flow_from_directory(
    directory=train_path,
    target_size=(tile_size,tile_size),
    color_mode="rgb",  # rgb for color
    batch_size=batch_size,
    shuffle=False,
    class_mode=class_mode,  # 'sparse' for multiclass, 'binary' for binary
    subset='validation'
)

Found 608896 images belonging to 4 classes.
Found 152224 images belonging to 4 classes.


### Load and compile network

In [7]:
if class_mode == "binary":
    metrics = [
        tf.keras.metrics.BinaryCrossentropy(name="bce"),  # BinaryCrossentropy for binary
        tf.keras.metrics.BinaryAccuracy(name="acc")
    ]
    base_loss = tf.keras.losses.binary_crossentropy
    final_layer_node = 1
    binary = True
    
elif class_mode == "sparse":
    metrics = [
        tf.keras.metrics.SparseCategoricalCrossentropy(name="sce"), 
        tf.keras.metrics.SparseCategoricalAccuracy(name="acc")
    ]
    base_loss = tf.keras.losses.sparse_categorical_crossentropy
    final_layer_node = t_flow.num_classes
    binary = False

elif class_mode == "categorical":
    metrics = [
        tf.keras.metrics.CategoricalCrossentropy(name="cce"),
        tf.keras.metrics.CategoricalAccuracy(name="acc")
    ]
    base_loss = tf.keras.losses.categorical_crossentropy
    final_layer_node = t_flow.num_classes
    binary = False
    
else:
    print("no class mode provided")
    
    
base_model = bm.create_thesis_model(tile_size=tile_size, channels=3, final_layer_node=final_layer_node)
i = tf.keras.Input(shape=(tile_size, tile_size, channels))
x_i = base_model(i)
cst_model = cst.CSTModel(inputs=i, outputs=x_i, alpha=alpha, n_st_components=n_st_components, 
                         dist_layers=dist_layer, binary=binary)


cst_model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3, amsgrad=True),
    loss = base_loss,
    metrics = metrics 
)


### Train network

In [8]:

cst_model.run_eagerly = False

cst_model.fit(
    x=t_flow,
    validation_data=v_flow,
    epochs=epochs,
    class_weight=bm.get_class_weights(t_flow.classes)
)

Epoch 1/10


2023-07-05 15:41:06.101072: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-07-05 15:41:06.408001: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100


   23/19028 [..............................] - ETA: 2:38:35 - loss_0: 1.9582 - cst_metric: 2.0683 - sce_: 1.3053 - acc_: 0.3378 - sce_0: 1.3509 - acc_0: 0.2814 - sce_1: 1.3017 - acc_1: 0.3292 - sce_2: 1.3148 - acc_2: 0.3711 - sce_3: 1.2982 - acc_3: 0.3279

KeyboardInterrupt: 

In [ ]:
cst_model.dist_layers[0].__dict__

In [ ]:
cst_model.compiled_loss._loss_metric.__dict__

In [ ]:
cst_model.compiled_metrics._metrics[0][0].__dict__

In [ ]:
metrics = [
        tf.keras.metrics.BinaryCrossentropy(name="bce"),  # BinaryCrossentropy for binary
        tf.keras.metrics.BinaryAccuracy(name="acc")
    ]

base_model = bm.create_thesis_model(tile_size=tile_size, channels=3, final_layer_node=1)
i = tf.keras.Input(shape=(tile_size, tile_size, channels))
x_i = base_model(i)
cst_model = cst.CSTModel(inputs=i, outputs=x_i, alpha=alpha, n_st_components=n_st_components, 
                         dist_layers=dist_layer, binary=True)


cst_model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3, amsgrad=True),
    loss = tf.keras.losses.binary_crossentropy,
    metrics = metrics 
)

cst_model.fit(
    x=t_flow,
    validation_data=v_flow,
    epochs=epochs,
    class_weight=bm.get_class_weights(t_flow.classes)
)

In [ ]:
cst_model.dist_layers[0].__dict__

In [ ]:
ly.RandomBrightness(**dcs["brightness"]).__dict__